## First We Read the Data from the URL

In [ ]:
# Reading data from url
import pandas as pd
import requests
from bs4 import BeautifulSoup

#url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/NewLinkWebscrapingHints.md'
#url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969).'
page=requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")


## Now We Create the table from the URL

In [41]:
## **Firstly create a list**  
table_contents=[]

table=soup.find('table')


# Let's start the webscrapping of the table

In [48]:
for row in table.findAll('td'):
#    print(row.text, 'DONE')
#    continue
    ## Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**
    cell = {}

    ## As postal code contains upto 3 characters extract that using tablerow.p.text**
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]

         ## Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.
        cell['Borough'] = (row.span.text).split('(')[0]
        #print (row.text)
        #continue
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')

         ## Append to the list**  
        table_contents.append(cell)

#print(table_contents)


In [49]:
## Create a dataframe with list**
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


# Finally we print the shape of the resulting dataframe

In [66]:
df.shape

(206, 3)

## Now we continue opening the Geospatial data

In [62]:
df_geodata = pd.read_csv('Geospatial_Coordinates.csv')
df_geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Here we merge both dataframes using the Postal Code

In [70]:
df_1 = df.rename(columns={'PostalCode': 'Postal Code'})
             
df_complete = pd.merge(df_1, df_geodata, on="Postal Code")
df_complete.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M4A,North York,Victoria Village,43.725882,-79.315572
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
8,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
9,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
